# Normalize LIMS taxa

In [151]:
import sys
sys.path.append('../../../')
import glob
import re
import os.path
import hashlib

import pandas as pd
import numpy as np

from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR

from scripts.normalize_data import (
    csv_cleanup,
    update_metadata,
    get_taxonomy_columns,
    clean_taxon_name,
    get_non_taxa_fields,
    remove_whitespace
)
import scripts.normalize_taxa as nt


In [152]:
# metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 
metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes_4.csv'

normalized_fields_path =  OUTPUT_DIR/'normalized_data'/f'eODP_unified_data_structure_2022_02_21.csv'


clean_data_path = CLEAN_DATA_DIR

additional_taxa_path = RAW_DATA_DIR/'PI_processed_files'/'LIMS_Micropal_CSV_4_taxa_ADDTL_TAXA.csv'


date = '2022-04-28'

crosswalk_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_crosswalk_{date}.csv"
taxa_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_list_{date}.csv"


In [153]:
dex_sin_taxa = {
    "Dextral:Sinistral _N. acostaensis_": ['Neogloboquadrina acostaensis (dextral)', 'Neogloboquadrina acostaensis (sinistral)'],
    "Dextral:Sinistral _P. finalis_": ['Pulleniatina finalis (dextral)', 'Pulleniatina finalis (sinistral)'],
    "Dextral:Sinistral _P. obliquiloculata_": ['Pulleniatina obliquiloculata (dextral)', 'Pulleniatina obliquiloculata (sinistral)'],
    "Dextral:Sinistral _P. praecursor_": ['Pulleniatina praecursor (dextral)', 'Pulleniatina praecursor (sinistral)'],
    "Dextral:Sinistral _P. praespectabilis_": ['Pulleniatina praespectabilis (dextral)', 'Pulleniatina praespectabilis (sinistral)'],
    "Dextral:Sinistral _P. primalis_": ['Pulleniatina primalis (dextral)', 'Pulleniatina primalis (sinistral)'],
    "Dextral:Sinistral _P. spectabilis_": ['Pulleniatina spectabilis (dextral)', 'Pulleniatina spectabilis (sinistral)']
}

In [154]:
def log_df(df, row_count=5):
    print(df.shape)
    return df.head(row_count)


## Clean up taxa values

Look for taxa columns that have "code [extra text]", and remove "[extra text]".

In [155]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols,add_expedition_section_cols
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False,True
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False,True
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False,False


In [156]:
taxa_df = pd.read_csv(crosswalk_file, dtype=str)
taxa_df.head()


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments,additional species comments,eodp_id
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group,NaN,0
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group,NaN,1
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams,Others,NaN,not a taxa,Andy,other benthic foraminifera,NaN,2
3,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pleurostomellidae indet.,benthic_forams,Pleurostomellids comment,NaN,not a taxa,Andy,NaN,NaN,3
4,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ostracoda indet.,benthic_forams,Ostracoda spp.,Ostracoda spp.,group abundance,"Summer 2020: go in as Ostracoda indet, and the...",NaN,NaN,4


In [157]:
taxa_list = set(taxa_df['verbatim_name'].unique())
len(taxa_list)

5256

In [158]:
def clean_up_taxa_values(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    
    taxa_cols = list(set(content.columns).intersection(taxa_list))
    filter_df = content[taxa_cols]
    content[list(taxa_cols)] = filter_df.replace(to_replace =' *\[.*\] *', value = '', regex = True) 
    
    # get rid of NAs in order to compare two dataframes   
    changed = not filter_df.fillna('').equals(content[taxa_cols].fillna(''))

    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [clean_up_taxa_values(file) for file in metadata['path']] 



### Update metadata

In [159]:
dict = {"clean_up_taxa_values": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols,add_expedition_section_cols,clean_up_taxa_values
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False,True,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False,True,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False,False,False


In [160]:
new_metadata.to_csv(metadata_file, index=False)

## Clean up non-taxa metadata values

Look for non-taxa metadata columns that have "code [extra text]", and remove "[extra text]".

In [161]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols,add_expedition_section_cols,clean_up_taxa_values
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False,True,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False,True,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False,False,False


In [162]:
cols = ['normalized', 'taxa_317-present']
non_taxa_df = pd.read_csv(normalized_fields_path, dtype=str, header=5, usecols=cols)
non_taxa_df.head()


,normalized,taxa_317-present
0,Sample,Label ID; Sample
1,Expedition,Exp
2,Site,Site
3,Hole,Hole
4,Core,Core


In [163]:
non_taxa_dict = get_non_taxa_fields(non_taxa_df, 'taxa_317-present')
non_taxa_list = set(non_taxa_dict.keys())

In [164]:
non_taxa_list

{'% Planktic Foraminifera within whole sample',
 'A/W',
 'Abundance',
 'Additional zone name',
 'Additional zone name (short)',
 'Age',
 'Aspect comment (etching)',
 'BF Group abundance',
 'BF Preservation',
 'BF comment',
 'BF preservation',
 'Bottom Depth [m]',
 'Bottom Depth[m] [m]',
 'Bottom [cm]',
 'Bottom depth [m]',
 'Bottom offset [cm]',
 'Bottom[cm] [cm]',
 'COMMENTS',
 'Comment',
 'Comment (general)',
 'Comments',
 'Core',
 'Datum age average [Ma]',
 'Datum age maximum (Ma)',
 'Datum age minimum (Ma)',
 'Datum author year',
 'Datum name',
 'Datum name generic',
 'Datum region',
 'Datum type',
 'Diatom abundance',
 'Diatom preservation dissolution',
 'Diatom preservation fragmentation',
 'Diatoms group abundance',
 'Exp',
 'Extra Sample ID Data',
 'Foram abundance',
 'General comment',
 'Genus/species (upper zone)',
 'Genus/species lower zone)',
 'Group Abundance',
 'Group abundance',
 'Group abundance (%)',
 'Group preservation',
 'Hole',
 'Label ID',
 'Large Benthic Forams [

In [165]:
def clean_up_taxa_meta_values(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    
    nontaxa_cols = list(set(content.columns).intersection(non_taxa_list))
    filter_df = content[nontaxa_cols]
    content[list(nontaxa_cols)] = filter_df.replace(to_replace =' *\[.*\] *', value = '', regex = True) 
    
    # get rid of NAs in order to compare two dataframes   
    changed = not filter_df.fillna('').equals(content[nontaxa_cols].fillna(''))

    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [clean_up_taxa_meta_values(file) for file in metadata['path']] 

### Update metadata

In [166]:
dict = {"clean_up_taxa_metadata_values": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols,add_expedition_section_cols,clean_up_taxa_values,clean_up_taxa_metadata_values
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False,True,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False,True,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False,False,False,False


In [167]:
new_metadata.to_csv(metadata_file, index=False)

## Add eodp_id

In [168]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols,add_expedition_section_cols,clean_up_taxa_values,clean_up_taxa_metadata_values
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False,True,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False,True,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False,False,False,False


In [169]:
def add_eodp_id(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path, dtype=str)
    content.dropna(how='all', axis='rows', inplace=True)
    
    def add_hash_id(id):
        return hashlib.md5(f'{file}{id}'.encode()).hexdigest()
            
    content['eodp_id'] = content.index 
    content['eodp_id'] = content['eodp_id'].apply(add_hash_id)
    
    
    content.to_csv(path, index=False)

res = [add_eodp_id(file) for file in metadata['path']]

## split Dextral:Sinistral taxa

split  Dextral:Sinistral column into two columns.

In [170]:
cols = ['normalized_name', 'verbatim_name', 'non-taxa descriptor']
taxa_df = pd.read_csv(crosswalk_file, dtype=str, usecols=cols)
taxa_df = taxa_df[taxa_df.duplicated(subset=['verbatim_name'])]
taxa_df = taxa_df[taxa_df['non-taxa descriptor'].notna()]

taxa_df

,non-taxa descriptor,normalized_name,verbatim_name
3090,sinistral,Neogloboquadrina acostaensis (sinistral),Dextral:Sinistral _N. acostaensis_
3092,sinistral,Pulleniatina finalis (sinistral),Dextral:Sinistral _P. finalis_
3094,sinistral,Pulleniatina obliquiloculata (sinistral),Dextral:Sinistral _P. obliquiloculata_
3096,sinistral,Pulleniatina praecursor (sinistral),Dextral:Sinistral _P. praecursor_
3098,sinistral,Pulleniatina praespectabilis (sinistral),Dextral:Sinistral _P. praespectabilis_
3100,sinistral,Pulleniatina primalis (sinistral),Dextral:Sinistral _P. primalis_
3102,sinistral,Pulleniatina spectabilis (sinistral),Dextral:Sinistral _P. spectabilis_


In [171]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols,add_expedition_section_cols,clean_up_taxa_values,clean_up_taxa_metadata_values
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False,True,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False,True,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False,False,False,False


In [172]:
def split_dextral_sinistral(file):
    path = f"{clean_data_path}/{file}"

    changed = False
    df_0 = pd.read_csv(path, dtype=str, nrows=0)
    
    file_target_taxa = set(df_0.columns).intersection(set(dex_sin_taxa.keys()))
    if len(file_target_taxa) > 0:
        changed = True
        df = pd.read_csv(path, dtype=str)
        for taxon in file_target_taxa:
            new_columns = dex_sin_taxa[taxon]
            split_columns = df[taxon].str.split(':', expand=True)
            if split_columns.shape[1] == 1:
                for col in new_columns:
                    df[col] = np.nan
            else:
                df[new_columns] = split_columns
            
            del df[taxon]

        df = csv_cleanup(df, path)
        df.to_csv(path, index=False)

    return changed 
            

change_columns = [split_dextral_sinistral(file) for file in metadata['path']]

In [173]:
dict = {"split_dextral_sinistral": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols,add_expedition_section_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False,True,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False,True,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False,False,False,False,False


In [174]:
new_metadata.to_csv(metadata_file, index=False)

## clean up  additional species file
 add path and normalized_name columns

In [77]:
metadata = pd.read_csv(metadata_file, dtype=str)
metadata.head()

,file,path,taxon_groups,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,363-U1482A-Benthic_Forams.csv,LIMS/Micropal_CSV_1/363-U1482A-Benthic_Forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,True,False
1,320_U1336A_Nannofossils_2.csv,LIMS/Micropal_CSV_1/320_U1336A_Nannofossils_2.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
2,375_U1518F_planktic_forams.csv,LIMS/Micropal_CSV_1/375_U1518F_planktic_forams...,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
3,320_U1334A_Nannofossils_1.csv,LIMS/Micropal_CSV_1/320_U1334A_Nannofossils_1.csv,nannofossils,False,False,False,True,False,False,False,True,False,False,False,False,False
4,318_U1358B_Palynology.csv,LIMS/Micropal_CSV_1/318_U1358B_Palynology.csv,palynology,False,False,False,False,False,False,False,True,False,False,False,False,False


In [78]:
df = pd.read_csv(additional_taxa_path, dtype=str)
log_df(df)

(30, 36)


,sample_id,file,sample,taxon_group,verbatim_name,code,Name,Comment,Notes (change to Internal only notes?),Any taxon above genus,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lonchosphaera spicata,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Poulpus spp.-like,NaN,Poulpus spp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lithostrobus cuspidatus,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Gondwanaria dogieli,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Tetraplecta pinigera/Euscenium corynephorum,NaN,Plagiacanthidae indet.,NaN,NaN,Plagiacanthidae indet.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
set(df['taxon_group'])

{'planktic_forams', 'radiolarians'}

In [80]:
for index, row in df.iterrows():
    path = metadata[metadata['file'] == row['file']]['path'].values[0]
    df.loc[df['file'] == row['file'], 'path'] = path
    
    nt.add_normalized_name_column(df)

log_df(df)

(30, 38)


,sample_id,file,sample,taxon_group,verbatim_name,code,Name,Comment,Notes (change to Internal only notes?),Any taxon above genus,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,path,normalized_name
0,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lonchosphaera spicata,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Lonchosphaera spicata
1,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Poulpus spp.-like,NaN,Poulpus spp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Poulpus spp.
2,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lithostrobus cuspidatus,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Lithostrobus cuspidatus
3,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Gondwanaria dogieli,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Gondwanaria dogieli
4,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Tetraplecta pinigera/Euscenium corynephorum,NaN,Plagiacanthidae indet.,NaN,NaN,Plagiacanthidae indet.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Plagiacanthidae indet.


In [81]:
df.to_csv(additional_taxa_path, index=False)

## add additional species taxa to crosswalk taxa list

In [123]:
existing_crosswalk = pd.read_csv(crosswalk_file, dtype=str)
log_df(existing_crosswalk, 3)
# 5250

(5250, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams,Others,NaN,not a taxa,Andy,other benthic foraminifera


In [124]:
df = pd.read_csv(additional_taxa_path, dtype=str)
log_df(df)

(30, 38)


,sample_id,file,sample,taxon_group,verbatim_name,code,Name,Comment,Notes (change to Internal only notes?),Any taxon above genus,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,path,normalized_name
0,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lonchosphaera spicata,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Lonchosphaera spicata
1,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Poulpus spp.-like,NaN,Poulpus spp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Poulpus spp.
2,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lithostrobus cuspidatus,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Lithostrobus cuspidatus
3,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Gondwanaria dogieli,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Gondwanaria dogieli
4,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Tetraplecta pinigera/Euscenium corynephorum,NaN,Plagiacanthidae indet.,NaN,NaN,Plagiacanthidae indet.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Plagiacanthidae indet.


set verbatim_name to normalized_name since we are using normalized_name as the columns names

In [125]:
df['verbatim_name'] = df['normalized_name']

In [126]:
fields = nt.taxa_rank_fields + nt.taxa_fields + nt.metadata_fields 
fields

['Any taxon above genus',
 'genus modifier',
 'genus name',
 'subgenera modifier',
 'subgenera name',
 'species modifier',
 'species name',
 'subspecies modifier',
 'subspecies name',
 'non-taxa descriptor',
 'normalized_name',
 'taxon_group',
 'verbatim_name',
 'name comment field',
 'Comment',
 'Notes (change to Internal only notes?)',
 'comments']

In [127]:
filtered_taxa = pd.DataFrame(df, columns=fields)
log_df(filtered_taxa, 2)

(30, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,NaN,NaN,Lonchosphaera,NaN,NaN,NaN,spicata,NaN,NaN,NaN,Lonchosphaera spicata,radiolarians,Lonchosphaera spicata,NaN,NaN,NaN,NaN
1,NaN,NaN,Poulpus,NaN,NaN,NaN,spp.,NaN,NaN,NaN,Poulpus spp.,radiolarians,Poulpus spp.,NaN,NaN,NaN,NaN


In [128]:
filtered_taxa = filtered_taxa.drop(filtered_taxa[filtered_taxa['normalized_name'] == ''].index)
log_df(filtered_taxa)

filtered_taxa.drop_duplicates(keep='first', inplace=True, subset =['verbatim_name', 'normalized_name'])
log_df(filtered_taxa)

(30, 17)
(20, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,NaN,NaN,Lonchosphaera,NaN,NaN,NaN,spicata,NaN,NaN,NaN,Lonchosphaera spicata,radiolarians,Lonchosphaera spicata,NaN,NaN,NaN,NaN
1,NaN,NaN,Poulpus,NaN,NaN,NaN,spp.,NaN,NaN,NaN,Poulpus spp.,radiolarians,Poulpus spp.,NaN,NaN,NaN,NaN
2,NaN,NaN,Lithostrobus,NaN,NaN,NaN,cuspidatus,NaN,NaN,NaN,Lithostrobus cuspidatus,radiolarians,Lithostrobus cuspidatus,NaN,NaN,NaN,NaN
3,NaN,NaN,Gondwanaria,NaN,NaN,NaN,dogieli,NaN,NaN,NaN,Gondwanaria dogieli,radiolarians,Gondwanaria dogieli,NaN,NaN,NaN,NaN
4,Plagiacanthidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plagiacanthidae indet.,radiolarians,Plagiacanthidae indet.,NaN,NaN,NaN,NaN


In [129]:
existing_crosswalk.columns == filtered_taxa.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [130]:
combine_df = pd.concat([existing_crosswalk, filtered_taxa])
remove_whitespace(combine_df)

combine_df.shape

(5270, 17)

In [131]:
combine_df.drop_duplicates(keep='first', inplace=True, subset =['verbatim_name', 'normalized_name'])
log_df(combine_df)
# 5264

(5264, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,,,Euuvigerina,,,,miozea,,,,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group
1,,,Euuvigerina,,,,rodleyi,,,,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group
2,Foraminifera indet.,,,,,,,,,,Foraminifera indet.,benthic_forams,Others,,not a taxa,Andy,other benthic foraminifera
3,Pleurostomellidae indet.,,,,,,,,,,Pleurostomellidae indet.,benthic_forams,Pleurostomellids comment,,not a taxa,Andy,
4,Ostracoda indet.,,,,,,,,,,Ostracoda indet.,benthic_forams,Ostracoda spp.,Ostracoda spp.,group abundance,"Summer 2020: go in as Ostracoda indet, and the...",


In [132]:
comments_df = df[['additional species comments', 'verbatim_name']].copy()
comments_df.drop_duplicates(inplace=True)
log_df(comments_df)

(20, 2)


,additional species comments,verbatim_name
0,"Taxa which are ""Indeterminate due to data sour...",Lonchosphaera spicata
1,"Taxa which are ""Indeterminate due to data sour...",Poulpus spp.
2,"Taxa which are ""Indeterminate due to data sour...",Lithostrobus cuspidatus
3,"Taxa which are ""Indeterminate due to data sour...",Gondwanaria dogieli
4,"Taxa which are ""Indeterminate due to data sour...",Plagiacanthidae indet.


In [134]:
combine_df = combine_df.merge(comments_df, how='left')
log_df(combine_df)
# 5264

(5264, 18)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments,additional species comments
0,,,Euuvigerina,,,,miozea,,,,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group,NaN
1,,,Euuvigerina,,,,rodleyi,,,,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group,NaN
2,Foraminifera indet.,,,,,,,,,,Foraminifera indet.,benthic_forams,Others,,not a taxa,Andy,other benthic foraminifera,NaN
3,Pleurostomellidae indet.,,,,,,,,,,Pleurostomellidae indet.,benthic_forams,Pleurostomellids comment,,not a taxa,Andy,,NaN
4,Ostracoda indet.,,,,,,,,,,Ostracoda indet.,benthic_forams,Ostracoda spp.,Ostracoda spp.,group abundance,"Summer 2020: go in as Ostracoda indet, and the...",,NaN


In [135]:
combine_df.to_csv(crosswalk_file, index=False)

## add additional species taxa to taxa list

In [136]:
existing_taxa = pd.read_csv(taxa_list_file, dtype=str)
log_df(existing_taxa, 3)
# 4662

(4662, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212476,Rhizaria


In [137]:
df = pd.read_csv(additional_taxa_path, dtype=str)
log_df(df)

(30, 38)


,sample_id,file,sample,taxon_group,verbatim_name,code,Name,Comment,Notes (change to Internal only notes?),Any taxon above genus,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,path,normalized_name
0,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lonchosphaera spicata,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Lonchosphaera spicata
1,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Poulpus spp.-like,NaN,Poulpus spp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Poulpus spp.
2,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lithostrobus cuspidatus,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Lithostrobus cuspidatus
3,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Gondwanaria dogieli,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Gondwanaria dogieli
4,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Tetraplecta pinigera/Euscenium corynephorum,NaN,Plagiacanthidae indet.,NaN,NaN,Plagiacanthidae indet.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Plagiacanthidae indet.


In [138]:
fields = nt.taxa_rank_fields  + nt.taxa_fields + nt.pdbd_fields
fields

['Any taxon above genus',
 'genus modifier',
 'genus name',
 'subgenera modifier',
 'subgenera name',
 'species modifier',
 'species name',
 'subspecies modifier',
 'subspecies name',
 'non-taxa descriptor',
 'normalized_name',
 'taxon_group',
 'pbdb_taxon_id',
 'pbdb_taxon_name',
 'pbdb_taxon_rank',
 'family_taxon_id',
 'family_taxon_name',
 'order_taxon_id',
 'order_taxon_name',
 'class_taxon_id',
 'class_taxon_name',
 'phylum_taxon_id',
 'phylum_taxon_name',
 'kingdom_taxon_id',
 'kingdom_taxon_name']

In [139]:
filtered_taxa = pd.DataFrame(df, columns=fields)
log_df(filtered_taxa, 2)

(30, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,NaN,NaN,Lonchosphaera,NaN,NaN,NaN,spicata,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Poulpus,NaN,NaN,NaN,spp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
filtered_taxa = filtered_taxa.drop(filtered_taxa[filtered_taxa['normalized_name'] == ''].index)
log_df(filtered_taxa)

filtered_taxa.drop_duplicates(keep='first', inplace=True, subset =[ 'normalized_name', 'taxon_group'])
log_df(filtered_taxa)


(30, 25)
(20, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,NaN,NaN,Lonchosphaera,NaN,NaN,NaN,spicata,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Poulpus,NaN,NaN,NaN,spp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Lithostrobus,NaN,NaN,NaN,cuspidatus,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Gondwanaria,NaN,NaN,NaN,dogieli,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Plagiacanthidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
existing_taxa.columns == filtered_taxa.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [142]:
combine_df = pd.concat([existing_taxa, filtered_taxa])
remove_whitespace(combine_df)
combine_df.shape

# 4682

(4682, 25)

In [143]:
combine_df.drop_duplicates(inplace=True, subset =[ 'normalized_name', 'taxon_group'] )
log_df(combine_df)
# 4676

(4676, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Euuvigerina,,,,miozea,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
1,,,Euuvigerina,,,,rodleyi,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
2,Foraminifera indet.,,,,,,,,,,...,,,,,,,,,212476,Rhizaria
3,Pleurostomellidae indet.,,,,,,,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
4,Ostracoda indet.,,,,,,,,,,...,,,,,,,18891,Arthropoda,325038,Animalia


In [144]:
combine_df.to_csv(taxa_list_file, index=False)

## add additional species taxa to data files 

In [175]:
df = pd.read_csv(additional_taxa_path, dtype=str)
log_df(df)

(30, 38)


,sample_id,file,sample,taxon_group,verbatim_name,code,Name,Comment,Notes (change to Internal only notes?),Any taxon above genus,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,path,normalized_name
0,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lonchosphaera spicata,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Lonchosphaera spicata
1,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Poulpus spp.-like,NaN,Poulpus spp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Poulpus spp.
2,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Lithostrobus cuspidatus,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Lithostrobus cuspidatus
3,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Gondwanaria dogieli,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Gondwanaria dogieli
4,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,Tetraplecta pinigera/Euscenium corynephorum,NaN,Plagiacanthidae indet.,NaN,NaN,Plagiacanthidae indet.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv,Plagiacanthidae indet.


In [176]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_bad_characters,remove_empty_rows,remove_spaces,delete_duplicate_rows,delete_duplicate_columns,update_sample_col,update_top_bottom,add_missing_cols,add_expedition_section_cols,clean_up_taxa_values,clean_up_taxa_metadata_values,split_dextral_sinistral
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,True,True,False,True,False,False,False,False,False,True,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,True,False,False,False,False,False,True,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,False,True,False,False,True,False,False,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,True,False,False,True,False,False,False,False,False,False


In [180]:
for index, row in df.iterrows():
    tmp_df = metadata[metadata['file'] == row['file']]
    if len(tmp_df) > 0:
        path = tmp_df['path'].values[0]
        temp_df = pd.read_csv(clean_data_path/path, dtype=str)

        temp_df.loc[temp_df['Sample'] == row['sample'], row['normalized_name']] = 'Indeterminate due to data source'

        temp_df = csv_cleanup(temp_df, path)
        temp_df.to_csv(clean_data_path/path, index=False)


## add eodp_id to taxa crosswalk

In [145]:
df = pd.read_csv(crosswalk_file, dtype=str)
log_df(df, 3)
# 5264

(5264, 18)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments,additional species comments
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group,NaN
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group,NaN
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams,Others,NaN,not a taxa,Andy,other benthic foraminifera,NaN


In [146]:
df['eodp_id'] = df.index
log_df(df, 3)


(5264, 19)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments,additional species comments,eodp_id
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>100 m group,NaN,0
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",>50 m group,NaN,1
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams,Others,NaN,not a taxa,Andy,other benthic foraminifera,NaN,2


In [147]:
df.to_csv(crosswalk_file, index=False)